In [2]:
import pandas as pd
import ast
from collections import Counter
import re
from sklearn.model_selection import train_test_split
import ast
from functools import reduce
from collections import defaultdict
import spacy
nlp = spacy.load("en_core_web_sm")
from collections import Counter

In [5]:
df = pd.read_csv('Recipe_dataset/NLGRecipe_KownCountriesOnly_Train.csv')

In [7]:
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER,countries
0,639161,Mexican Dish,"[""1 can whole kernel corn"", ""1 can chili"", ""1 ...","[""Mix in baking dish."", ""Bake 30 to 45 minutes...",www.cookbooks.com/Recipe-Details.aspx?id=766171,Gathered,"[""whole kernel corn"", ""chili"", ""tamales"", ""wat...",Mexico
1,241058,Spanish Rice,"[""4 c. boiled rice"", ""3 onions, chopped"", ""4 T...","[""Brown onions in hot cooking fat."", ""Combine ...",www.cookbooks.com/Recipe-Details.aspx?id=361539,Gathered,"[""boiled rice"", ""onions"", ""cooking fat"", ""clov...",Spain
2,1954593,Baked Mexican Pinwheels,"[""1 cup Philadelphia Light Cream Cheese Produc...","[""Heat oven to 375F."", ""Mix cream cheese produ...",www.kraftrecipes.com/recipes/baked-mexican-pin...,Recipes1M,"[""Cream Cheese"", ""Cheese"", ""chicken"", ""tomato""...",Mexico
3,2184694,Michelle Naranjos Indian Bread,"[""7 cups all-purpose flour"", ""1/2 teaspoon sal...","[""In a large bowl, combine the flour and salt ...",www.cookstr.com/recipes/michelle-naranjorsquos...,Recipes1M,"[""flour"", ""salt"", ""lard"", ""active dry yeast"", ...",India
4,940912,Grilled Eggplant Stack With Greek Kasseri Cheese,"[""1 teaspoon salt, or as needed"", ""2 eggplants...","[""Lightly sprinkle salt over eggplant slices a...",www.allrecipes.com/recipe/231641/grilled-eggpl...,Gathered,"[""salt"", ""eggplants"", ""olive oil"", ""garlic"", ""...",Greece


In [9]:
list_countries = list(set(df['countries']))

In [11]:
stopword_ingredients = ['sugar', 'milk', 'salt', 'water', 'pepper', 'butter', 'oil', 'olive oil', 'vegetable oil', 'garlic', 'onion', 
                        'egg', 'eggs', 'flour', 'brown sugar', 'sugar', 'vinegar', 'powdered sugar', 'accompaniment','style', 'meal', 'ground', 'white', 'cream', 
                        'cheese', 'fryer', 'chunky', 'soup', 'thighs', 'wheat', 'shell', 'bell', 'sauce', 'vegetable', 'shortening', 'bun', 'grain', 'powder', 
                        'yeast', 'pwdr','purpose','ketchup','noodle','bouillon', 'cube','lasagna', 'hamburger', 'bread', 'soda','breadcrumb', 'mayo',
                        'confectioner','broth','macaroni','elbow','shell', 'yolk', 'buttermilk', 'dressing', 'meat', 'bun', 'mayonnaise', 'skinless',
                        'whipping', 'miracle', 'white', 'stock', 'pasta', 'rice', 'crust', 'seasoning', 'bouillon', 'juice', 'leaf', 'breast', ' breast', 'chuck', 'velveeta',
                        'granule' 'evoo', 'deli', 'roast', 'roll', 'seed',  'black',  'pizza','head', 'green', 'stew', 'dill', 'weed', 'cube', 'stalk', 'cooking spray', 
                        'spray', 'cooking', 'tenderloin', 'chicken breast', 'paste', 'spice', 'allspice', 'worcestershire', 'chicken breast', 'tomato paste', 'mustard', 
                        'extract', 'unsalted', 'ingredient', 'party','dark', 'flower', 'brazil', 'wine', 'beef', 'chicken','cake', 'canola', 'vanilla', 'tomato', 'shrimp']
authrorized_pos= ['PROPN', 'PRON', 'NOUN']

In [13]:
def text_cleaning(recipe_list):

    clean_list = []
    for i in range(len(recipe_list)):
        recipe_doc = nlp(str(recipe_list[i]))
        temp_list = []
        for token in recipe_doc:
            if token.text.lower() not in stopword_ingredients:
                if len(token.text.lower()) > 3:
                    if token.pos_ in authrorized_pos:
                        temp_list.append(token.lemma_.lower())
        if len(temp_list) > 0:
            clean_text = ' '.join(temp_list)
            clean_list.append(clean_text)
    return clean_list

In [27]:
Dict_ = defaultdict(list)
for country in list_countries:
    df_country = df[df['countries'] == country]
    list_ingredients = list(df_country['NER'])
    list_temp = []
    for str_list in list_ingredients:
        list_temp.extend(ast.literal_eval(str_list))
    
    count_ingredient = Counter(list_temp)
    #print(len(count_ingredient))
    avg = sum(count_ingredient.values()) / float(len(count_ingredient))
   #print('avg :', avg)
    filtered_ingredient = dict((k, v) for k, v in count_ingredient.items() if v >= avg+1.0) #+1.0 on the avg because most average are around 1 or 2
   # print(len(filtered_ingredient))
    #print('=======================')
    set_ingredient = list(filtered_ingredient.keys())
    clean_set = text_cleaning(set_ingredient)
    clean_set = list(set(clean_set))
    final_list = []
    for elem in clean_set:
        if elem not in stopword_ingredients:
            final_list.append(elem)
    final_list = list(set(final_list))
    if country.lower() in Dict_:
        Dict_[country.lower()].extend(final_list)
    else:
        Dict_[country.lower()] = final_list

In [28]:
Dict_['south korea']

['sriracha',
 'marinade',
 'nut',
 'cayenne',
 'peanut',
 'apple',
 'hoisin',
 'gochujang',
 'olive',
 'corn syrup',
 'tamari',
 'recipe',
 'pork',
 'flank steak',
 'radish',
 'fish',
 'carrot',
 'rib',
 'corn tortilla',
 'pear',
 'cinnamon',
 'brown',
 'cabbage',
 'sake',
 'fresh ginger',
 'honey',
 'kiwi',
 'shallot',
 'apple cider',
 'spinach',
 'sesame',
 'daikon radish',
 'anchovy',
 'baby spinach',
 'mushroom',
 'cucumber',
 'sprout',
 'scallion',
 'sesame seed',
 'bean sprout',
 'oyster',
 'skirt steak',
 'sherry',
 'maple syrup',
 'pork shoulder',
 'cornstarch',
 'celery',
 'zucchini',
 'choy',
 'chestnut',
 'ginger',
 'cilantro',
 'mirin',
 'spring onion',
 'ginger root',
 'potato',
 'lemon',
 'firm tofu',
 'bacon',
 'wonton wrappers',
 'spring',
 'chili',
 'lime']

In [29]:
def jaccard_similarity(list1, list2):
    #print(len(list1), len(list2))
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return float(intersection) / union

In [ ]:
list_countries

In [63]:
threshold = 0.3
Dict_sim_countries = defaultdict(list)
for i, country in enumerate(list_countries):
    list_ingredient_country = Dict_[country.lower()]
    list_other_countries = list_countries[:i] + list_countries[i+1:]
    temp_list = []
    if len(list_ingredient_country) > 5:
        for other in list_other_countries:
            list_ingredient_other = Dict_[other.lower()]
            #if len(list_ingredient_other) > 0: 
             #   intersection = len(list(set(list_ingredient_country).intersection(list_ingredient_other)))
                #print(intersection)
            #    if intersection / len(list_ingredient_country) >= threshold:
           #         temp_list.append(other)
            if len(list_ingredient_other) > 0: 
                sim_score = jaccard_similarity(list_ingredient_country, list_ingredient_other)
                if sim_score >= threshold:
                    temp_list.append(other)
        Dict_sim_countries[country.lower()].extend(temp_list)

In [ ]:
Dict_sim_countries

In [108]:
import json 

In [65]:
with open("country_similarities_large_2.json", "w") as outfile: 
    json.dump(Dict_sim_countries, outfile)